In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import cuda
from tqdm import tqdm
from transformers import BertConfig
from transformers import BertModel
import json
import matplotlib.pyplot as plt
import os
import pickle

from transformers import AutoTokenizer, T5ForConditionalGeneration

PRETRAINED_MODEL_NAME = "t5-small"
EPOCHS = 10
TRAIN_BATCH_SIZE = 256
VALID_BATCH_SIZE = 256
SEQ_LEN=17

model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(
    PRETRAINED_MODEL_NAME,
    padding_side="left"
)
tokenizer_r = AutoTokenizer.from_pretrained(
    PRETRAINED_MODEL_NAME,
    padding_side="right"
)

train_data = pd.read_csv("new_17resid_train_data.txt", encoding='unicode_escape', names=['Seq'])
train_target = pd.read_csv("Ntrain_3states_target.txt", encoding='unicode_escape', names=['target'])

test_data = pd.read_csv("new_17resid_test_data.txt", encoding='unicode_escape', names=['Seq'])
test_target = pd.read_csv("Ntest_3states_target.txt", encoding='unicode_escape', names=['target'])

df = pd.concat([train_data, train_target], axis=1)
df_ = pd.concat([test_data, test_target], axis=1)

df["Seq"] = df.Seq.apply(lambda val: " ".join([x.replace("B", tokenizer.pad_token) for x in val]))
df_["Seq"] = df_.Seq.apply(lambda val: " ".join([x.replace("B", tokenizer.pad_token) for x in val]))

device = 'cuda' if cuda.is_available() else 'cpu'

/media/HDD/bportelli/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [2]:
from torch.utils.data import TensorDataset, DataLoader

def batchify(l, batch_size):
    i = 0
    while i < len(l):
        i += batch_size
        yield l[i-batch_size:i]

def to_tensor_dataset(data, tokenizer, kind):
    
    input_ids = []
    labels = []
    
    outputs = tokenizer_r(["Helyx", "Strand", "Coil"], padding="longest")
    outputs = {i:v for i,v in enumerate(outputs.input_ids)}
    
    labels = data.target.apply(lambda x: outputs[x])#[outputs[row.target] for _,row in tqdm(data.iterrows(), total=len(data))]
    labels = torch.tensor(labels)
    
    print(labels.shape)
        
    batch_size = 200
    
    for sentences in tqdm(batchify(data.Seq, batch_size), total=len(data)//batch_size):
        
        tok_out = tokenizer(sentences.tolist(), add_special_tokens=False, padding="longest")
        tok_out = [
            [x for x in seq if (x != 3) and (x != 0)]
            for seq in tok_out.input_ids
        ]
        tok_out = [
            seq + [tokenizer.pad_token_id]*(SEQ_LEN-len(seq))
            for seq in tok_out
        ]
        
        input_ids += tok_out
    
    input_ids = torch.tensor(input_ids)
    
    print(input_ids.shape)
    
    return TensorDataset(input_ids, labels)

import pickle
import os

if not os.path.exists("train_dataset.pkl"):

    train_dataset = to_tensor_dataset(df, tokenizer, "TRAIN")
    test_dataset = to_tensor_dataset(df_, tokenizer, "TEST")
    test_dataloader = DataLoader(test_dataset, batch_size=8)

    with open("train_dataset.pkl", "wb") as f:
        pickle.dump(train_dataset, f)
    with open("test_dataset.pkl", "wb") as f:
        pickle.dump(test_dataset, f)

torch.Size([5672821, 4])


28365it [02:08, 220.34it/s]                                                                                             


torch.Size([5672821, 17])
torch.Size([490612, 4])


2454it [00:10, 237.62it/s]                                                                                              


torch.Size([490612, 17])


In [3]:
with open("train_dataset.pkl", "rb") as f:
    train_dataset = pickle.load(f)
with open("test_dataset.pkl", "rb") as f:
    test_dataset = pickle.load(f)

In [4]:
import transformers

training_args = transformers.TrainingArguments(
    # ------------------------------------------------------- [epochs and batch size]
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=1,
    # ------------------------------------------------------- [hyperparams]
    warmup_steps=100, 
    weight_decay=0.01,
    # ------------------------------------------------------- [save and logging]
    output_dir=".", 
    overwrite_output_dir = True,
    do_eval = False,
    logging_strategy="epoch", # activate if interested
    save_strategy="no",
    save_total_limit = None,
    # -------------------------------------------------------
)
trainer = transformers.Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    data_collator = lambda data: {
        'input_ids': torch.stack([f[0] for f in data]), 
        # 'attention_mask': torch.stack([f[1] for f in data]), 
        'labels': torch.stack([f[1] for f in data]),
    }
)

In [5]:
trainer.train()

/media/HDD/bportelli/env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5672821
  Num Epochs = 20
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 443200


Step,Training Loss



KeyboardInterrupt

